#**Trabalho final - Simulador de ChatBot com GenAI e VectorDB**

**Nota de atenção:**
- Leia com atenção o descritivo do trabalho e as orientações do template.
- O trabalho deve ser entregue **respeitando a estrutura do arquivo de template**, utilizando o notebook "Template Trabalho final - Simulador de ChatBot.ipynb" e compactado no formato .zip.
- Deve haver apenas um arquivo no formato .ipynb, consolidando todo o trabalho.

**Participantes (RM - NOME):**<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

###**Caso de uso - Marketplace de classificados veículos**

Imagine que você trabalha na empresa **iAutos** que tem como produto principal um marketplace de classificados de veículos e você como um Engenheiro de Dados, tem a missão de ajudar a empresa a oferecer um melhor serviço para seus clientes (vendedores e compradores).

Contexto:
- Sua empresa oferece um serviço (site/plataforma) de classificados de veículos (semelhantes aos marketplaces convencionais, mas focado em vendas de veículos), onde os vendedores (sellers) podem cadastrar e anunciar seus veículos, e compradores (buyers) podem buscar veículos de seu interesse e contatar os vendedores para negociar os veículos de seu interesse.
- Diariamente a empresa recebe muitos chamados de dúvidas sobre as regras de publicação de veículos e regras de uso do produto.
- Veja mais detalhes das regras do documento de [Quem Somos e Políticas de Uso](https://drive.google.com/file/d/1-ZpUOl8OA4lxa8CJ6auT42hSxaF3jclk) (em PDF).

---
Como podemos ajudar a empresa?


###**Desafio**

A ideia é criar um ChatBot mais "turbinado" do que praticamos nas aulas.
Esse ChatBot será ser responsável por atender e responder as dúvidas dos clientes sobre o marketplace.

Criem um **ChatBot demonstrativo** usando IA Generativa para criar um novo **canal de atendimento** para tirar dúvidas dos clientes referente a plataforma, políticas de uso e publicação, para isso considere as orientações abaixo:

- Utilizem o framework do **LangChain** para criar a lógica do ChatBot, gerenciar a conexão com o modelo LLM e modelo de Embedding, para gerenciar o ChromaDB com base no contexto e para gerenciar a memória do agente.
- Usem como contexto o arquivo PDF [Quem Somos e Políticas de Uso](https://drive.google.com/file/d/1-ZpUOl8OA4lxa8CJ6auT42hSxaF3jclk).
- Criem o VectorDB com o ChromaDB e com base no contexto do PDF.
  - Obs.: Não necessariamente precisa carregar o PDF, fiquem a vontade para definirem a melhor estratégia.
- Utilizem as boas práticas de Prompt Engineering para criar o template do prompt para o serviço e gerenciar a conversa.
- Utilizem como base e referência todos os materiais apresentados, tanto de Generative AI quanto de Database for GenAI.
  - Estruturem bem o trabalho, organizem em funções, expliquem e documentem bem os códigos e decisões.
- Fiquem a vontade de complementar o contexto e o prompt para otimizar o serviço.
- Fiquem a vontade de trazer técnicas que façam sentido e complemente o trabalho.

### Dicas:
- Comecem os desenvolvimentos de forma simples, testando os componentes e vá aumentando a complexidade gradativamente.
- Utilizem os modelos LLMs da OpenAI ou da Azure com OpenAI.

###**Orientações:**

---
####**Usem o Google Colab com Python e esse template para desenvolverem o trabalho.**
---

**1. Desenvolvimento e testes**, nessa parte é onde vocês podem explorar o desenvolvimento do trabalho aplicando as técnicas, testando as ferramentas e serviços de GenAI.
  - Explorem diferentes formas de tratar o problema. Comecem testando os componentes e definindo a melhor estratégia.
  - Testem as ferramentas, framework, APIs e técnicas de prompt engineering.
  - Fiquem à vontade para explorar os serviços e frameworks vistos em aula: API da OpenAI Platform, API da Azure AI Foundry, LangChain e outros.
  - Sejam criativos, mas não precisa de muita complexidade e podem explorar outras formas de desenvolver.
  - Expliquem as decisões e racional do desenvolvimento. **Abuse dos comentários**.

**2. Processo final**, aqui nessa parte separem apenas o processo final com um pipeline completo para o ChatBot, desde a instalação das bibliotecas até a simulação.
  - Organizem em funções que façam sentido.
  - Resultado esperado: Um processo estruturado utilizando LangChain e ChromaDB, criando um VectorDB com uma boa estratégia de indexação e busca (retriever), uma simulação do ChatBot (pode ser estruturando uma API ou alguma lógica para simular uma conversa).
  - Exemplos: Deixem e/ou compartilhem no notebook exemplos de conversas.
  - Testem bem esse pipeline antes, pois o professor tentará executar o processo para validar a implementação.

###**Avaliação:**
O trabalho será avaliado pelas seguintes diretrizes:
  - Demonstração de conhecimento com os temas abordados em sala de aula.
  - Utilização correta dos frameworks, APIs e aplicação das técnicas de prompt engineering.
  - Organização, comentários e explicação certamente vão ajudar na nota.
  - Resultado esperado seguindo as orientações do professor nesse template.

###**Atenção:**
- Usem a conta da **Azure AI Foundry** ou da **OpenAI Platform** para desenvolverem o trabalho, mas dê preferência para a conta da Azure por causa dos limites de crédito. **Não deixem suas credenciais no trabalho, por favor!**
- Trabalhos iguais são passíveis de reprovação ou desconto de nota.
- Respeitem a estrutura do template fornecido pelo professor.
- Limite de 2 a 4 pessoas por grupo, de preferência o mesmo grupo do Startup One.

##**1. Desenvolvimento e testes**

Desenvolva aqui:

In [ ]:
import os
from dotenv import load_dotenv

KEY_NGROK = "" # @param {"type":"string"}

os.environ["KEY_NGROK"] = KEY_NGROK

In [2]:
import os, sys, time, psutil, socket, subprocess, shutil

CHROMA_HOST = "127.0.0.1" 
CHROMA_PORT = 8000

def is_port_in_use(port, host="127.0.0.1"):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(0.5)
        return s.connect_ex((host, port)) == 0

def wait_for_port(port, host="127.0.0.1", timeout=30):
    start = time.time()
    while time.time() - start < timeout:
        if is_port_in_use(port, host):
            return True
        time.sleep(0.5)
    return False

def build_chroma_cmd(host, port):
        chroma_exe = os.path.join(os.path.dirname(sys.executable), "Scripts", "chroma.exe")
        return [chroma_exe, "run", "--host", host, "--port", str(port)]

In [3]:
chroma_exe = os.path.join(os.path.dirname(sys.executable), "Scripts", "chroma.exe")

In [4]:
chroma_exe

'c:\\Users\\JOAO PC\\AppData\\Local\\Programs\\Python\\Python313\\Scripts\\chroma.exe'

In [5]:
os.path.exists(chroma_exe)

True

In [13]:
is_port_in_use(CHROMA_PORT)


False

In [ ]:
chroma = build_chroma_cmd(CHROMA_HOST, CHROMA_PORT)

In [18]:
chroma

['c:\\Users\\JOAO PC\\AppData\\Local\\Programs\\Python\\Python313\\Scripts\\chroma.exe',
 'run',
 '--host',
 '127.0.0.1',
 '--port',
 '8000']

In [19]:
subprocess.Popen(chroma, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)


<Popen: returncode: None args: ['c:\\Users\\JOAO PC\\AppData\\Local\\Program...>

In [20]:
wait_for_port(CHROMA_PORT)

True

In [34]:
# Verifica se o servidor está respondendo (opcional, mas recomendado)
print("Verificando status do servidor...")
!curl http://127.0.0.1:8000/api/v2/heartbeat
print("\nSe a resposta acima for '{\"nanosecond heartbeat\":...}', o servidor está no ar!")

Verificando status do servidor...
{"nanosecond heartbeat":1761933964922155600}

Se a resposta acima for '{"nanosecond heartbeat":...}', o servidor está no ar!


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    44  100    44    0     0  29810      0 --:--:-- --:--:-- --:--:-- 44000


In [22]:
# Associar o Servidor Chroma com o Ngrok
import time
from pyngrok import ngrok, conf

# Configurando e iniciando o Ngrok
print("Configurando e iniciando o Ngrok")
ngrok.set_auth_token(KEY_NGROK)
#conf.get_default().auth_token = KEY_NGROK

# Mata qualquer túnel anterior para garantir um início limpo
try:
  ngrok.kill()
except:
  pass

# Expor o servidor com Ngrok
public_url = ngrok.connect(8000)
print(f"Servidor Chroma está online e acessível na URL: {public_url}")
# Inicia o túnel na porta 8000, onde nosso servidor Chroma está escutando

Configurando e iniciando o Ngrok
Servidor Chroma está online e acessível na URL: NgrokTunnel: "https://operculate-vernon-unmissed.ngrok-free.dev" -> "http://localhost:8000"


In [23]:
public_url.public_url

'https://operculate-vernon-unmissed.ngrok-free.dev'

In [24]:
url = public_url.public_url  

# testa o endpoint de heartbeat
!curl {url}/api/v2/heartbeat
print("\nSe a resposta acima for '{\"nanosecond heartbeat\":...}', o servidor está no ar! 🚀")


{"nanosecond heartbeat":1761933402510016300}

Se a resposta acima for '{"nanosecond heartbeat":...}', o servidor está no ar! 🚀


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0    44    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    44  100    44    0     0     62      0 --:--:-- --:--:-- --:--:--    62


##**2. Processo final**

In [25]:
## LangChain
!pip install langchain==0.3.27 langchain_community==0.3.27 langchain-openai==0.3.27 --quiet

## ChromaDB
!pip install langchain-chroma==0.2.5 chromadb-client==1.0.20 --quiet

## Ngrok
#!pip install pyngrok==7.3.0 --quiet

## Outras
!pip install fastapi==0.116.1 uvicorn==0.35.0 nest_asyncio==1.6.0 --quiet

In [26]:
# Bibliotecas e configuração do ambiente
import os
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import uuid
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import chromadb

# Bibliotecas LanChain
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Ativa o modo debug do LangChain (para ver o que acontece por baixo do capô!)
flg_log_debug = False
langchain.debug = flg_log_debug

In [ ]:
import requests
import os

url = "https://drive.google.com/file/d/1xZIpwP1JggimbZrtGLnuaTf6DrhbAS9T/view?usp=sharing"

file_id = url.split("/d/")[1].split("/")[0]
url_download = f"https://drive.google.com/uc?export=download&id={file_id}"

file_name = "contexto_rag_chatbot.txt"

response = requests.get(url_download)
response.raise_for_status()

with open(file_name, 'wb') as f:
    f.write(response.content)

arquivo_contexto = file_name
with open(arquivo_contexto, 'r', encoding='utf-8') as file:
    contexto = file.read()

print(contexto[:500])

#**Quem Somos?**
Bem-vindo à AutoToya Veículos, sua concessionária oficial da Toyota!

Desde o início, nosso compromisso é oferecer a você uma experiência completa e personalizada, indo além da venda de veículos. Na AutoToya, você encontrará uma ampla variedade de modelos Toyota, desde os mais recentes lançamentos até veículos seminovos de alta qualidade.

Na AutoToya, estamos comprometidos em oferecer a você uma experiência completa e personalizada. Contamos com uma ampla variedade de modelos T


In [29]:
## Por quantidade de tokens
import tiktoken

# Escolhe o tokenizer utilizando o mesmo método do modelo de Embeddings
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

# Função que conta tokens
def fn_conta_token(text: str) -> int:
    return len(encoding.encode(text))

# Configura o splitter medindo por tokens
text_splitter_token = RecursiveCharacterTextSplitter(
    chunk_size = 500,         # máximo de tokens por chunk
    chunk_overlap = 50,       # sobreposição de tokens por chunk
    length_function = fn_conta_token
)

# O método create_documents() divide o contexto nos chunks e já o formata em um objeto tipo Document do LangChain.
documents = text_splitter_token.create_documents([contexto])
print(f"O contexto foi dividido em {len(documents)} documentos (chunks).")

O contexto foi dividido em 12 documentos (chunks).


In [ ]:
OPENAI_API_KEY = ""

In [32]:
# Define qual o modelo de embeddings, no caso vamos usar um modelo da OpenAI - text-embedding-3-small
embedding_model = OpenAIEmbeddings(
    openai_api_key = OPENAI_API_KEY,
    model = 'text-embedding-3-small'
)

C:\Users\JOAO PC\AppData\Local\Temp\ipykernel_4580\114160704.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


In [35]:
client_chromadb = chromadb.HttpClient(
    host = CHROMA_HOST
)

In [36]:
# O nome da "coleção" no servidor. Pense nisso como o nome de uma tabela em um banco de dados.
CHROMADB_COLLECTION_NAME = "chromadb_vactorstore_autotoya"

# Cria o Vectorstore com o Chroma a partir dos documentos divididos
vectorstore = Chroma.from_documents(
    documents = documents,
    embedding = embedding_model,
    client = client_chromadb,  # Passamos o "client" em vez do persist_directory
    collection_name = CHROMADB_COLLECTION_NAME
)

# Instancia o retriever (podemos controlar o número de chunks do retorno da análise de similaridade)
retriever = vectorstore.as_retriever(
    search_type = "similarity",   # similarity por "mmr" - (max marginal relevance) - pode trazer mais diversidade nos resultados
    search_kwargs = {"k": 3}      # define quantos chunks vão retornar
)

In [37]:
# Teste da busca por similaridade
query = "O que é a AutoToya?"

docs_retornados = vectorstore.similarity_search(query, k=2)
print(docs_retornados[0].page_content)
print(docs_retornados[1].page_content)

#**Quem Somos?**
Bem-vindo à AutoToya Veículos, sua concessionária oficial da Toyota!

Desde o início, nosso compromisso é oferecer a você uma experiência completa e personalizada, indo além da venda de veículos. Na AutoToya, você encontrará uma ampla variedade de modelos Toyota, desde os mais recentes lançamentos até veículos seminovos de alta qualidade.

Na AutoToya, estamos comprometidos em oferecer a você uma experiência completa e personalizada. Contamos com uma ampla variedade de modelos Toyota, desde os mais recentes lançamentos até veículos seminovos de alta qualidade, além de preços exclusivos para taxistas, pessoas com deficiência (PCD), produtores rurais e vendas diretas para CNPJ.

Também cuidamos do seu Toyota como ninguém. Oferecemos peças originais e serviços especializados para garantir a máxima segurança, desempenho e durabilidade do seu veículo.

Na AutoToya, acreditamos que escolher um carro é mais do que uma compra, é um momento especial. Venha nos visitar, conheça 

In [38]:
# Configura o chat prompt template com System Rules - regras fixas do assistente
system_template = """
Você é um assistente virtual altamente inteligente e atende clientes de uma loja (concessionária) de venda de veículos chamada AutoToya Veículos.

#**Seu papel e objetivo é:**
1. Responder dúvidas sobre a loja, veículos e serviços oferecidos pela loja.
2. Qualificar e engajar o cliente para aumentar o interesse nos veículos (priorize os veículos zero km, mas não perca a venda.)
3. Convidar o cliente para fazer um test-drive e conhecer o veículo pessoalmente.
4. Ajudar o cliente a agendar a visita de test-drive.

##**Orientações e regras que deve seguir:**
- Seu nome é ToyaBot.
- Responda sempre em Português (pt-br)
- Seja simpático e prestativo. Use emojs para deixar a conversa divertida, como: 🤖 🚗 👊 💪 🚀 😊 🪄
- Responda sempre de forma educada e clara.
- Nunca destratar um cliente sendo rude ou arrogante por exemplo.
- Use o contexto fornecido para responder à pergunta de forma coerente.
- **Mantenha uma coerência nas respostas com base no histórico do chat.**
- Se o contexto não for suficiente, peça mais detalhes para o cliente.
- Responda **APENAS** perguntas que estão no contexto, se não souber diga que não pode responder (respnda de forma cordial) e interaja voltando para o contexto da conversa e seu objetivo.

Contexto:
{context}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),   ## regras fixas do assistente
    MessagesPlaceholder(variable_name="chat_history", n_messages = 6, optional = True), ## Insere o histórico de conversas aqui (opcional e limitado a N mensagens) / Monta uma lista de mensagens (AIMessage, HumanMessage, SystemMessage)
    HumanMessagePromptTemplate.from_template("{question}")        ## define a entrada do usuário - entrada dinâmica
])
# optional = True, deixa o histórico como opcional caso esteja vazio para evitar erros


In [40]:
# Modelo LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    openai_api_key=OPENAI_API_KEY
)


C:\Users\JOAO PC\AppData\Local\Temp\ipykernel_4580\155446263.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [41]:
# Configura o memory para manter o histórico da conversa
memory_test = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
    )

# Cria a cadeia de conversa com retriever e memória
chain_test = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory_test,
    combine_docs_chain_kwargs = {"prompt": chat_prompt},
    chain_type = "stuff", # O tipo "stuff" simplesmente junta os chunks em um único prompt.
)


C:\Users\JOAO PC\AppData\Local\Temp\ipykernel_4580\2282799827.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_test = ConversationBufferMemory(


In [42]:
# Função com o loop de iteração simulando um chat
def fn_chat_test():
  """Função para simular um chat com o assistente"""
  print("Iniciado o Chatbot da AutoToya! Digite 'sair' para encerrar.\n")

  # Pré-carregar uma mensagem de histórico com uma saudação inicial
  memory_test.chat_memory.clear() ## limpa o histórico (buffer)
  memory_test.chat_memory.add_ai_message("Olá! Seja bem-vindo. Sou um assistente virtual, me chamo ToyaBot 🤖. \nComo posso ajudar você hoje? Está em busca de informações sobre algum veículo ou serviço da AutoToya Veículos? 🚗✨")
  print(f"ToyaBot 🤖: {memory_test.chat_memory.messages[0].content}")

  while True:
    user_question = input("\nVocê: ")
    if user_question.lower() == "sair":
        print("ToyaBot 🤖: 👋 Conversa encerrada. Até logo! 🚗")
        break
    response = chain_test.invoke({"question": user_question})
    #response = chain_test.run(question = user_question) ## Essa função está depreciada
    print(f"ToyaBot 🤖: {response['answer']}")


In [43]:
flg_log_debug = False

if __name__ == "__main__":
    langchain.debug = flg_log_debug
    fn_chat_test()


Iniciado o Chatbot da AutoToya! Digite 'sair' para encerrar.

ToyaBot 🤖: Olá! Seja bem-vindo. Sou um assistente virtual, me chamo ToyaBot 🤖. 
Como posso ajudar você hoje? Está em busca de informações sobre algum veículo ou serviço da AutoToya Veículos? 🚗✨
ToyaBot 🤖: A AutoToya Veículos é a sua concessionária oficial da Toyota! 🚗 Desde o início, nosso compromisso é oferecer uma experiência completa e personalizada, indo além da venda de veículos. Na AutoToya, você encontrará uma ampla variedade de modelos Toyota, desde os mais recentes lançamentos até veículos seminovos de alta qualidade.

Estamos dedicados a cuidar do seu Toyota, oferecendo peças originais e serviços especializados para garantir a segurança, desempenho e durabilidade do seu veículo. Além disso, contamos com preços exclusivos para taxistas, pessoas com deficiência (PCD), produtores rurais e vendas diretas para CNPJ.

Na AutoToya, acreditamos que escolher um carro é mais do que uma compra, é um momento especial. Venha no

In [44]:
# Gerenciamento de múltiplos usuários (memória por sessão)
sessions = {}

def get_chat_chain(session_id):
    if session_id not in sessions:

        # Configura o memory para manter o histórico da conversa
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
            )

        # Cria a cadeia de conversa com retriever e memória
        chain = ConversationalRetrievalChain.from_llm(
            llm = llm,
            retriever = retriever,
            memory = memory,
            combine_docs_chain_kwargs = {"prompt": chat_prompt},
            chain_type = "stuff", # O tipo "stuff" simplesmente junta os chunks em um único prompt.
            )

        # Carrega mensagem inicial
        memory.chat_memory.add_ai_message("Olá! Seja bem-vindo. Sou um assistente virtual, me chamo ToyaBot 🤖. \nComo posso ajudar você hoje? Está em busca de informações sobre algum veículo ou serviço da AutoToya Veículos? 🚗✨")

        # Gerencia sessão - cada chat vira um objeto chain identificado pelo session_id
        sessions[session_id] = chain
    return sessions[session_id]

# Configura o FastAPI para criar a estrutura de API
app = FastAPI(title="API - Chatbot ToyaBot", version="1.0")

# Estrutura resposta com parse do pydantic
class ChatRequest(BaseModel):
    session_id: str
    question: str

# Rota para chamar a função que chama o LLM e toda estrutura do chat (chain, retriever, vectorstore, etc.)
@app.post("/chat")
def chat(req: ChatRequest):
    chain = get_chat_chain(req.session_id)
    #resposta = chain.run(question = req.question)
    resposta = chain.invoke({"question": req.question})
    resposta = resposta['answer']
    return {"answer": resposta}

# Rota para inicializar o chat com a mensagem de saudação inicial
@app.get("/start/{session_id}")
def start(session_id: str):
    chain = get_chat_chain(session_id)
    msg_inicial = chain.memory.chat_memory.messages[0].content
    return {"answer": msg_inicial}

# Rota principal que renderiza página HTML do chat (simples)
@app.get("/", response_class = HTMLResponse)
def index():
    return """
<html>
<head><title>🤖 ToyaBot Chat 🤖</title></head>
<body style="font-family:Arial">
  <h2>🤖 ToyaBot - AutoToya Veículos 🤖</h2>
  <div id="chat" style="border:1px solid #ccc; padding:10px; width:400px; height:300px; overflow-y:scroll;"></div>
  <input type="text" id="msg" style="width:300px;" placeholder="Digite sua mensagem..."/>
  <button onclick="send()">Enviar</button>
  <script>
    let session_id = Math.random().toString(36).substring(7);

    async function startChat(){
      let r = await fetch('/start/' + session_id);
      let data = await r.json();
      document.getElementById("chat").innerHTML += "<b>ToyaBot:</b> " + data.answer + "<br/>";
    }

    async function send(){
      let msg = document.getElementById("msg").value;
      document.getElementById("chat").innerHTML += "<b>Você:</b> " + msg + "<br/>";
      let r = await fetch('/chat', {
        method: 'POST',
        headers: {'Content-Type':'application/json'},
        body: JSON.stringify({session_id: session_id, question: msg})
      });
      let data = await r.json();
      document.getElementById("chat").innerHTML += "<b>ToyaBot:</b> " + data.answer + "<br/>";
      document.getElementById("msg").value="";
    }

    // inicia o chat chamando o backend
    startChat();
  </script>
</body>
</html>
"""


In [45]:
# Executa o App no Colab + ngrok
if __name__ == "__main__":
  #ngrok.set_auth_token(KEY_NGROK) ## Já fizemos essa configuração antes (agora é opcional)
  public_url = ngrok.connect(8001)
  print("URL pública gerada pelo Ngrok:", public_url)

  nest_asyncio.apply()
  uvicorn.run(app, host="0.0.0.0", port=8001, reload=False)

INFO:     Started server process [4580]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


URL pública gerada pelo Ngrok: NgrokTunnel: "https://operculate-vernon-unmissed.ngrok-free.dev" -> "http://localhost:8001"
INFO:     187.255.127.46:0 - "GET / HTTP/1.1" 200 OK
INFO:     187.255.127.46:0 - "GET /start/0lkk0q HTTP/1.1" 200 OK
INFO:     187.255.127.46:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     187.255.127.46:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     187.255.127.46:0 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4580]


In [46]:
# Finaliza sessão do Ngrok (Agente)
ngrok.kill()

Desenvolva aqui: